In [1]:
!pip install langchain
!pip install streamlit
!pip install PyPDF2
!pip install sentence-transformers

In [2]:
import os
from PyPDF2 import PdfReader
import streamlit as st
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

def process_text(text):
    #CharacterTextSplitter로 문자열 텍스트를 청크로 분할
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)

    #임베딩 처리 (텍스트 임베딩), 임베딩모델 HuggingFaceEmbeddings 모델을 사용
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    documents = FAISS.from_texts(chunks, embeddings)
    return documents

def main():   #streamlit을 이용한 웹사이트 생성
    st.title("PDF 요약하기")
    st.divider()
    try:
        os.environ["OPENAI_API_KEY"] = "sk"   #openai api 키 입력
    except ValueError as e:
        st.error(str(e))
        return

    pdf = st.file_uploader('PDF파일을 업로드해주세요.', type='pdf')

    if pdf is not None:
        pdf_reader = PdfReader(pdf)
        text = ""   
        for page in pdf_reader.pages:
            text += page.extract_text()

        documents = process_text(text)
        query = "업로드 된 PDF 파일의 내용을 약 3~5문장으로 요약해주세요."

        if query:
            docs = documents.similarity_search(query)
            llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
            chain = load_qa_chain(llm, chain_type='stuff')

            with get_openai_callback() as cost:
                response = chain.run(input_documents=docs, question=query)
                print(cost)

            st.subheader('--요약 결과--')
            st.write(response)


if __name__ == '__main__':
    main()
            

2025-11-13 11:01:15.094 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-13 11:01:15.848 
  command:

    streamlit run C:\Users\chogu\anaconda3\envs\11m\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-11-13 11:01:15.850 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-13 11:01:15.851 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-13 11:01:15.851 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-13 11:01:15.852 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-13 11:01:15.854 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-13 11:01:15.854 Thread 'MainTh